# **Scrapping Algorithm**

In [1]:
from bs4 import BeautifulSoup  # to parse html
from datetime import datetime
import os
import pandas as pd

# import requests  # to send http requests
from pip._vendor import requests
from time import (
    sleep,
)  # function that allows you to pause the program. helpful to spae out requests
from tqdm import tqdm

c:\Users\Luismi\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [75]:
# Define our url: https://www.failory.com/startups/germany

html_text = requests.get('https://www.failory.com/startups/germany', timeout=15)

In [76]:
# Getting the info from the beautifulSoup
soup = BeautifulSoup(html_text.content, 'html.parser')
startups = soup.find_all('h3')
lists = soup.find_all('ul')
startups = [str(i) for i in startups]
lists = [str(i) for i in lists]
lists = [i.replace('</li><li>', '</li>  <li>') for i in lists]

In [77]:
startups = [i.strip("<h3>/)1234567890 ") for i in startups]
startups

['Celonis',
 'TradeRepublic',
 'Ada Healt',
 'Delivery Hero',
 'uberall',
 'TIER Mobility',
 'Foodpanda',
 'Scalable Capital',
 'Trade Republic',
 'FlixBus',
 'wefox',
 'HelloFres',
 'SoundCloud',
 'Mambu',
 'Lilium',
 'Flink Food',
 'Razor Group',
 'Forto',
 'Sanity Group',
 'WunderMobility',
 'OroraTec',
 'YFood',
 'Habyt',
 'OneFootball',
 'parcelLab',
 'ELEMENT Insurance',
 'Contentful',
 'Spryker Systems',
 'ATAI Life Sciences',
 'Adjust',
 'TIER',
 'CoachHub',
 'Vivid Money',
 'DHL',
 'Infarm',
 'Solarisbank',
 'GetYourGuide',
 'TWAICE',
 'Isar Aerospace',
 'undred',
 'HomeToGo',
 'Personio',
 'QUIN',
 'Volocopter',
 'Holidu',
 'Avi Medical',
 'Grover',
 'Raccoon.World',
 'Cognigy GmbH',
 'commercetools',
 'Lumenaza',
 'Enpal',
 'Zeotap',
 'airfocus',
 'Morressier',
 'Bunc',
 'Alasco',
 'Zalando',
 'SimScale',
 'Choco',
 'AUTO1 Group',
 'Billie',
 'Finoa',
 'sevDesk',
 'Signavio',
 'Babbel',
 'Silexica',
 'Raisin',
 'IDnow',
 'Pricefx',
 'BRYTER',
 'Arabesque S-Ray',
 'Camunda',


In [78]:
#Cleaning all the html stuff
import re
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):  #Function given I'm working on disentangling what ".sub" does... to be continued
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext
lists = [cleanhtml(lists[i]) for i in range(len(lists))]

In [79]:
lists = [i.split("  ") for i in lists]

In [80]:
for i, k in enumerate(startups):
    lists[i].insert(0, k)

In [103]:
lists_dict = []
for i in lists:
    job_dict = {}
    job_dict['Company'] = i[0]
    for k in i:
        if 'City' in k:
            k_rep = k.replace("City: ", "")
            job_dict['City'] = k_rep
        elif 'Started in' in k:
            k_rep = k.replace("Started in: ", "")
            job_dict['Started_in'] = k_rep
        elif 'Founders:' in k:
            k_rep = k.replace("Started in: ", "")
            job_dict['Founders'] = k_rep
        elif 'Industries:' in k:
            k_rep = k.replace("Industries: ", "")
            job_dict['Industries'] = k_rep
        elif 'Number of employees:' in k:
            k_rep = k.replace("Number of employees: ", "")
            job_dict['Number_of_employees'] = k_rep
        elif 'Funding amount' in k:
            k_rep = k.replace("Funding amount: ", "")
            job_dict['Funding_amount'] = k_rep
        elif 'Number of funding rounds: ' in k:
            k_rep = k.replace("Number of funding rounds: ", "")
            job_dict['Number_of_funding_rounds'] = k_rep
        elif 'Number of investors: ' in k:
            k_rep = k.replace("Number of investors: ", "")
            job_dict['Number_of_investors'] = k_rep
        else:
            continue
    lists_dict.append(job_dict)

In [105]:
df = pd.DataFrame.from_dict(lists_dict)
df

,Company,City,Started_in,Founders,Industries,Number_of_employees,Funding_amount,Number_of_funding_rounds,Number_of_investors
0,Celonis,Munich,2011,"Founders: Alexander Rinke, Bastian Nominacher,...","Analytics, Business Intelligence, SaaS, Software",1001-5000,"$1,367,500,000",4,9
1,TradeRepublic,Berlin,NaN,NaN,NaN,51-100,"$87,201,448",NaN,NaN
2,Ada Healt,Berlin,2011,"Founders: Claire Novorol, Daniel Nathrath, Mar...","Apps, Artificial Intelligence, Health Care, Me...",101-250,"$159,497,394",4,10
3,Delivery Hero,Berlin,NaN,NaN,NaN,10001,"$5,144,433,803",NaN,NaN
4,uberall,Berlin,2012,"Founders: David Federhen, Florian Hubner, Josh...","Information Technology, Location Based Service...",101-250,"$170,800,000",8,7
...,...,...,...,...,...,...,...,...,...
295,Finanzcheck,Hamburg,NaN,NaN,NaN,251-500,"$53,267,667",NaN,NaN
296,Layer,Berlin,2019,"Founders: Constantin Schünemann, David Barnows...",Software,1-10,"€ 5,700,000",2,7
297,E-Farm.com,Hamburg,NaN,NaN,NaN,11-50,"$5,300,000",NaN,NaN
298,XAIN,Berlin,NaN,NaN,NaN,11-50,"$7,035,853",NaN,NaN
